In [2]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [7]:
""" First, reload the data """
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']    
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']    
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # hint to help gc free up memory
    print('Training info:', train_dataset.shape, train_labels.shape)
    print('Valid info:', valid_dataset.shape, valid_labels.shape)
    print('Test info:', test_dataset.shape, test_labels.shape)

Training info: (20000, 28, 28) (20000,)
Valid info: (10000, 28, 28) (10000,)
Test info: (10000, 28, 28) (10000,)


In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size*image_size)).astype(np.float32)
    # map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0, ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset,labels

train_dataset,train_labels = reformat(train_dataset,train_labels)
valid_dataset,valid_labels = reformat(valid_dataset,valid_labels)
test_dataset,test_labels = reformat(test_dataset,test_labels)

print('Training info:', train_dataset.shape, train_labels.shape)
print('Valid info:', valid_dataset.shape, valid_labels.shape)
print('Test info:', test_dataset.shape, test_labels.shape)

Training info: (20000, 784) (20000, 10)
Valid info: (10000, 784) (10000, 10)
Test info: (10000, 784) (10000, 10)


In [11]:
train_subset = 10000

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

## 标准梯度下降

In [13]:
num_steps = 801

def accuracy(predictions, labels):
    accuracy = (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1)) / predictions.shape[0])
    return accuracy
    
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step%100 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.f%%' % accuracy(predictions, train_labels[:train_subset,:]))
            print('Validation accuracy:%.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy:%1.f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.390762
Training accuracy: 12%
Validation accuracy:15.0%
Loss at step 100: 2.271149
Training accuracy: 72%
Validation accuracy:70.0%
Loss at step 200: 1.837423
Training accuracy: 75%
Validation accuracy:72.7%
Loss at step 300: 1.601017
Training accuracy: 76%
Validation accuracy:73.6%
Loss at step 400: 1.439419
Training accuracy: 77%
Validation accuracy:74.0%
Loss at step 500: 1.317456
Training accuracy: 77%
Validation accuracy:74.2%
Loss at step 600: 1.220926
Training accuracy: 78%
Validation accuracy:74.5%
Loss at step 700: 1.142293
Training accuracy: 78%
Validation accuracy:74.6%
Loss at step 800: 1.076795
Training accuracy: 79%
Validation accuracy:74.8%
Test accuracy:82%


## 随机梯度下降 stochastic gradient

In [15]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal(shape=(image_size*image_size, num_labels)))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if step%100==0:
            print('Minibatch Loss at step %d: %f' % (step, l))
            print('Minibatch Training accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Minibatch Validation accuracy:%.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy:%1.f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch Loss at step 0: 20.215975
Minibatch Training accuracy: 7%
Minibatch Validation accuracy:11.2%
Minibatch Loss at step 100: 2.679464
Minibatch Training accuracy: 70%
Minibatch Validation accuracy:70.7%
Minibatch Loss at step 200: 1.515105
Minibatch Training accuracy: 81%
Minibatch Validation accuracy:72.5%
Minibatch Loss at step 300: 2.073150
Minibatch Training accuracy: 69%
Minibatch Validation accuracy:74.1%
Minibatch Loss at step 400: 1.511905
Minibatch Training accuracy: 77%
Minibatch Validation accuracy:75.1%
Minibatch Loss at step 500: 1.509688
Minibatch Training accuracy: 77%
Minibatch Validation accuracy:74.9%
Minibatch Loss at step 600: 1.674663
Minibatch Training accuracy: 77%
Minibatch Validation accuracy:74.9%
Minibatch Loss at step 700: 1.258005
Minibatch Training accuracy: 75%
Minibatch Validation accuracy:75.5%
Minibatch Loss at step 800: 1.138151
Minibatch Training accuracy: 78%
Minibatch Validation accuracy:76.3%
Minibatch Loss at step 900: 1.307070